In [1]:
using Flux
using Flux.Data: DataLoader
# using CUDA
using PyCall
using TOML
using CSV
using DataFrames
# using DataSets
using JLD
using DelimitedFiles
# using Plots
# using PlotThemes
using BSON: @save

In [2]:
# full_vocab = open(Vector{UInt8}, dataset("NikhilMukraj/smiles_vocab")) do buf
#     CSV.read(buf, DataFrame)
# end

# vocab = Matrix(full_vocab)

# buf = open(Vector{UInt8}, dataset("NikhilMukraj/preprocessed_d3_activity"))
# open("data.jld", "w") do f
#     write(f, buf)
# end

In [3]:
vocab = CSV.read("vocab.csv", DataFrame).tokens

71-element Vector{String7}:
 "#"
 "("
 ")"
 "."
 "/"
 "1"
 "2"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 ⋮
 "[Nd+3]"
 "[O-]"
 "[OH-]"
 "[P+]"
 "[Pt+2]"
 "[Pt]"
 "[S+]"
 "[S-]"
 "[Se]"
 "[Si]"
 "[Zn+2]"
 "\\"

In [4]:
data = load("data.jld")

Dict{String, Any} with 2 entries:
  "Y" => Float32[1.0 0.0; 1.0 0.0; … ; 0.0 1.0; 0.0 1.0]
  "X" => [[1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0…

In [5]:
X = data["X"]
Y = data["Y"]

18234×2 Matrix{Float32}:
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 ⋮    
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0

In [6]:
function trainTestSplit(X, Y; percent_train=.7, add_transpose=false)
    @assert size(X)[begin] == size(Y)[begin]

    train_length = floor(Int, size(X)[begin] * percent_train)

    if length(size(X)) == 1
        trainX = zeros(0,1)
        testX = zeros(0,1)
    else
        trainX = zeros(0,size(X)[end])
        testX = zeros(0,size(X)[end])
    end

    if length(size(Y)) == 1
        trainY = zeros(0,1)
        testY = zeros(0,1)
    else
        trainY = zeros(0,size(Y)[end])
        testY = zeros(0,size(Y)[end])
    end

    indices = collect(1:size(X)[begin])
    train_indices = []
    
    count = 0
    while count < train_length
        idx = rand(1:length(indices))
        push!(train_indices, indices[idx])
        deleteat!(indices, idx)
        count += 1
    end

    for i in train_indices
        trainX = vcat(trainX, X[i, :]')
        trainY = vcat(trainY, Y[i, :]')
    end

    for i in indices
        testX = vcat(testX, X[i, :]')
        testY = vcat(testY, Y[i, :]')
    end

    if add_transpose
        trainX = trainX'
        trainY = trainY'

        testX = testX'
        testY = testY'
    end

    return trainX, trainY, testX, testY
end

trainTestSplit (generic function with 1 method)

In [49]:
trainX[begin]
X[begin]

190×72 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [7]:
trainX, trainY, testX, testY = trainTestSplit(X, Y, percent_train=.9)
size(trainX), size(trainY), size(testX), size(testY)

((16410, 1), (16410, 2), (1824, 1), (1824, 2))

In [8]:
trainX = convert.(Matrix{Float32}, trainX)
trainY = convert(Matrix{Float32}, Float32.(trainY))
testX = convert.(Matrix{Float32}, testX)
testY = convert(Matrix{Float32}, Float32.(testY))

1824×2 Matrix{Float32}:
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 ⋮    
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0

In [9]:
trainX = reshape(trainX, 1, size(trainX)[begin])
trainY = trainY'
testX = reshape(testX, 1, size(testX)[begin])
testY = testY'

2×1824 adjoint(::Matrix{Float32}) with eltype Float32:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [10]:
N = length(vocab) + 1

model = Chain(
    LSTM(N, 128), relu,
    LSTM(128, 16), relu,
    Dense(16, 2), softmax
) # |> gpu

Chain(
  Recur(
    LSTMCell(72 => 128),                # 103_168 parameters
  ),
  NNlib.relu,
  Recur(
    LSTMCell(128 => 16),                # 9_312 parameters
  ),
  NNlib.relu,
  Dense(16 => 2),                       # 34 parameters
  NNlib.softmax,
)         # Total: 12 trainable arrays, 112_514 parameters,
          # plus 4 non-trainable, 288 parameters, summarysize 440.305 KiB.

In [11]:
function eval_model(x)    
    out = model(x)[:, end]
    Flux.reset!(model)

    return out'
end

function vcatTranspose(x)
    temp_matrix = Matrix{Float32}(undef, 0, size(x[begin])[end])

    for i in x 
        temp_matrix = vcat(temp_matrix, i)
    end

    return temp_matrix
end

function gpu_vcat(x)
    mat = zeros(size(x)[end], 2) |> gpu
    for i = 1:length(x)
        @inbounds mat[i, :] = Matrix(x[i])
    end
    
    return mat
end

function loss(x, y)
    pred = eval_model.(x)
    pred = vcatTranspose(pred)

    return Flux.crossentropy(pred', y)
end

function gpu_loss(x, y)
    pred = gpu_vcat(eval_model.(x))'

    return Flux.crossentropy(pred, y)
end

gpu_loss (generic function with 1 method)

In [31]:
function accuracy(ŷ, y)
    # check accuracy of prediction
    # ŷ
    # ŷ = eval_model(x)
    return sum(argmax.(eachrow(ŷ)) .== argmax.(eachrow(y))) / size(y)[begin]
end

function gpu_acc(ŷ, y)
    return sum(argmax.(eachrow(ŷ)) .== argmax.(eachrow(y))) / length(y)
end

gpu_acc (generic function with 1 method)

In [38]:
data = DataLoader((trainX, trainY), batchsize=32, shuffle=true)
opt = ADAM(0.1, (0.9, 0.999))

Adam(0.1, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [20]:
epochs = 1:100

training_losses = zeros(Float32, length(epochs))
testing_losses = zeros(Float32, length(epochs))
training_accs = zeros(Float32, length(epochs))
testing_accs = zeros(Float32, length(epochs))

100-element Vector{Float32}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [42]:
i = 3

println("Epoch: $i")
Flux.train!(loss, Flux.params(model), data, opt)
println("Finished train at epoch $i")

pred = eval_model.(trainX)
test_pred = eval_model.(testX)
pred = vcatTranspose(pred)
test_pred = vcatTranspose(test_pred)

training_losses[i] = Flux.crossentropy(pred', trainY)
testing_losses[i] = Flux.crossentropy(test_pred', testY)

training_accs[i] = accuracy(pred, trainY')
testing_accs[i] = accuracy(test_pred, testY')

println("Training Accuracy: $(training_accs[i])")
println("Testing Accuracy: $(testing_accs[i])")

println("Training Loss: $(training_losses[i])")
println("Testing Loss: $(testing_losses[i])")

Epoch: 3
Finished train at epoch 3
Training Accuracy: 0.499025
Testing Accuracy: 0.50877196
Training Loss: 0.6977161
Testing Loss: 0.6958887


In [15]:
epochs = 1:100

training_losses = zeros(Float32, length(epochs))
testing_losses = zeros(Float32, length(epochs))
training_accs = zeros(Float32, length(epochs))
testing_accs = zeros(Float32, length(epochs))

for i in epochs
    println("Epoch: $i")
    Flux.train!(loss, Flux.params(model), data, opt)
    println("Finished train at epoch $i")

    pred = eval_model.(trainX)
    test_pred = eval_model.(testX)
    pred = vcatTranspose(pred)
    test_pred = vcatTranspose(pred)

    training_losses[i] = Flux.crossentropy(pred', trainY)
    testing_losses[i] = Flux.crossentropy(test_pred', testY)

    training_accs[i] = accuracy(pred, trainY')
    testing_accs[i] = accuracy(test_pred, testY')

    println("Training Accuracy: $(training_accs[i])")
    println("Testing Accuracy: $(testing_accs[i])")

    println("Training Loss: $(training_losses[i])")
    println("Testing Loss: $(testing_losses[i])")
end

Epoch: 1
Finished train at epoch 1


LoadError: BoundsError: attempt to access Tuple{} at index [0]

In [16]:
pred = eval_model.(trainX)

1×16410 Matrix{LinearAlgebra.Adjoint{Float32, Vector{Float32}}}:
 [0.61571 0.38429]  [0.59434 0.40566]  …  [0.59434 0.40566]

In [ ]:
@save "rnn.bson" model

In [ ]:
writedlm("training_accs.txt", training_accs)
writedlm("testing_accs.txt", testing_accs)
writedlm("training_losses.txt", training_losses)
writedlm("testing_losses.txt", testing_losses)